In [77]:
from io import StringIO 
import pandas as pd
import requests

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

In [78]:
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3',
}

url = "https://stockanalysis.com/ipos/filings/"
response = requests.get(url, headers=headers)

ipo_filings = pd.read_html(StringIO(response.text))

In [79]:
ipos_df = ipo_filings[0]
ipos_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 329 entries, 0 to 328
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Filing Date     329 non-null    object
 1   Symbol          329 non-null    object
 2   Company Name    329 non-null    object
 3   Price Range     329 non-null    object
 4   Shares Offered  329 non-null    object
dtypes: object(5)
memory usage: 13.0+ KB


In [80]:
ipos_df.head()

,Filing Date,Symbol,Company Name,Price Range,Shares Offered
0,"Apr 26, 2024",EURK,Eureka Acquisition Corp,$10.00,5000000
1,"Apr 26, 2024",HDL,Super Hi International Holding Ltd.,-,-
2,"Apr 22, 2024",DRJT,Derun Group Inc,$5.00,-
3,"Apr 19, 2024",GPAT,GP-Act III Acquisition Corp.,$10.00,25000000
4,"Apr 16, 2024",JLJT,Jialiang Holdings Ltd,$5.00,-


In [81]:
# get date
ipos_df['Filing Date'] = pd.to_datetime(ipos_df['Filing Date'])

In [82]:
ipos_df.head()

,Filing Date,Symbol,Company Name,Price Range,Shares Offered
0,2024-04-26,EURK,Eureka Acquisition Corp,$10.00,5000000
1,2024-04-26,HDL,Super Hi International Holding Ltd.,-,-
2,2024-04-22,DRJT,Derun Group Inc,$5.00,-
3,2024-04-19,GPAT,GP-Act III Acquisition Corp.,$10.00,25000000
4,2024-04-16,JLJT,Jialiang Holdings Ltd,$5.00,-


In [83]:
ipos_df['Shares Offered'] = pd.to_numeric(ipos_df['Shares Offered'].astype(str).str.replace('-', ''))

In [84]:
ipos_df.tail()

,Filing Date,Symbol,Company Name,Price Range,Shares Offered
324,2020-01-21,GOXS,"Goxus, Inc.",$8.00 - $10.00,1500000.0
325,2020-01-21,UTXO,"UTXO Acquisition, Inc.",$10.00,5000000.0
326,2019-12-09,LOHA,Loha Co. Ltd,$8.00 - $10.00,2500000.0
327,2019-10-04,ZGHB,China Eco-Materials Group Co. Limited,$4.00,4300000.0
328,2018-12-27,FBOX,Fit Boxx Holdings Limited,$5.00 - $6.50,2000000.0


In [85]:
ipos_df['Price Range'] = ipos_df['Price Range'].astype(str).str.replace('$', '').str.replace('-', '')

In [86]:
def get_number_avg(string_range):
    numeric_range = [float(number) for number in string_range.split()]
    is_number = len(numeric_range) < 1 
    if is_number:
        return ''
    return sum(numeric_range) / len(numeric_range)

In [87]:
test = ''
print(get_number_avg(test))

In [88]:
ipos_df['Avg_price'] = pd.to_numeric(ipos_df['Price Range'].map(get_number_avg))

In [89]:
ipos_df.head()

,Filing Date,Symbol,Company Name,Price Range,Shares Offered,Avg_price
0,2024-04-26,EURK,Eureka Acquisition Corp,10.00,5000000.0,10.0
1,2024-04-26,HDL,Super Hi International Holding Ltd.,,NaN,NaN
2,2024-04-22,DRJT,Derun Group Inc,5.00,NaN,5.0
3,2024-04-19,GPAT,GP-Act III Acquisition Corp.,10.00,25000000.0,10.0
4,2024-04-16,JLJT,Jialiang Holdings Ltd,5.00,NaN,5.0


In [90]:
ipos_df['Shares_offered_value'] = ipos_df['Shares Offered'] * ipos_df['Avg_price'] 

In [91]:
ipos_df.head()

,Filing Date,Symbol,Company Name,Price Range,Shares Offered,Avg_price,Shares_offered_value
0,2024-04-26,EURK,Eureka Acquisition Corp,10.00,5000000.0,10.0,50000000.0
1,2024-04-26,HDL,Super Hi International Holding Ltd.,,NaN,NaN,NaN
2,2024-04-22,DRJT,Derun Group Inc,5.00,NaN,5.0,NaN
3,2024-04-19,GPAT,GP-Act III Acquisition Corp.,10.00,25000000.0,10.0,250000000.0
4,2024-04-16,JLJT,Jialiang Holdings Ltd,5.00,NaN,5.0,NaN


In [100]:
ipos_filtered = ipos_df[(ipos_df['Filing Date'] > '2023-01-01') & (ipos_df['Filing Date'] < '2023-12-31') & (ipos_df['Filing Date'].dt.dayofweek == 4)]

In [101]:
ipos_filtered.head()

,Filing Date,Symbol,Company Name,Price Range,Shares Offered,Avg_price,Shares_offered_value
52,2023-12-29,LEC,Lafayette Energy Corp,3.50 4.50,1200000.0,4.0,4800000.0
53,2023-12-29,EPSM,Epsium Enterprise Limited,,NaN,NaN,NaN
56,2023-12-22,LZMH,LZ Technology Holdings Limited,,NaN,NaN,NaN
57,2023-12-22,CHLW,Chun Hui Le Wan International Holding Group Ltd,,NaN,NaN,NaN
62,2023-12-15,GIT,Going International Holding Company Limited,,NaN,NaN,NaN


In [102]:
ipos_filtered['Shares_offered_value'].sum()

275950000.0